# Work out underlying alleles of DE genes

We have 50 DE genes of interest. These will either be in regions where the two lines are the same genetically, or possibly different. So lets find out

In [3]:
#import pandas as pd #Ok pandas not working for the moment

import pdb 


class Introgression_bed:
    
    def __init__(self,line):
        
        fields = line.strip().split("\t")
        self.line = line
        self.chromosome = fields[0]
        self.start = int(fields[1])
        self.end = int(fields[2])
        self.name = fields[3]
        self.allele = fields[4]

        

class Bedlist:
    
    def __init__(self):
        
        self.bedlist = []
    
    def __len__(self):
        
        return len(self.bedlist)
    
    def __getitem__(self,index):
        
        return self.bedlist[index]
        
    def append(self,item):
        
        self.bedlist.append(item) 
        
    def get_allele(self, chromosome, start, end):

        """Pulls out allele at a particular locus, denoted 
        by start and end"""

        allele = ""

        for bed in self.bedlist:
            if chromosome ==  bed.chromosome:
                if start > bed.start and end < bed.end:
                    allele = bed.allele

        if allele:
            return allele

        else:
            print("Error! alelle not found!")
            return allele



Int17_file = "/cluster/db/mecoulter/RNAseq2/introgression_viewer/Int_17-Barke.bed" #Chromosome, start, end, name, allele

Int52_file = "/cluster/db/mecoulter/RNAseq2/introgression_viewer/Int_52-Barke.bed"

#candidate_file = "/cluster/db/mecoulter/RNAseq2/RNAquant_analysis_result/candidate_gene_complete_info50.csv"
candidate_file = "/cluster/db/mecoulter/RNAseq2/RNAquant_analysis_result/candidate_gene_complete_info34_experiment2only.csv"

output = "/cluster/db/mecoulter/RNAseq2/RNAquant_analysis_result/candidate_gene_complete_info34experiment2only_alleles.csv"


Int17s = Bedlist()
Int52s = Bedlist()


for line in open(Int17_file):
    Int17s.append(Introgression_bed(line))

for line in open(Int52_file):
    Int52s.append(Introgression_bed(line))
        

trans_counts = set()
chr3H_counts = set()

with open(output, "w") as out:
    
    for n, line in enumerate(open(candidate_file)):
        #print(str(n))
        #print(line)
        if not n:# if first line ??
            out.write(line.strip())
            out.write(",124_17 allele, 124_52 allele\n")#Add two new columns
            continue
        
        fields = line.strip().split(",")
        gene = fields[1].strip('"')
        chromosome = fields[6].strip('"')
        start, end = [int(n) for n in fields[7:9]]
        #breakpoint()

        Int17_allele = Int17s.get_allele(chromosome, start, end)#Find allele for gene based on coordinates and the bed file
        Int52_allele = Int52s.get_allele(chromosome, start, end)
        
        if Int17_allele ==  Int52_allele:
            trans_counts.add(gene)
        if chromosome == "chr3H":
            chr3H_counts.add(gene)
            
        
        out.write(line.strip())
        out.write(f",{Int17_allele},{Int52_allele}\n")#Add two new columns

    
print(f"Number of DE genes that are definately trans regulated: {len(trans_counts)}\n")
print(f"Number of DE genes on 3H: {len(chr3H_counts)}\n")
print(f"Number of DE genes definatetively trans regulated and on 3H: {len(trans_counts & chr3H_counts)}")
    


Number of DE genes that are definately trans regulated: 6

Number of DE genes on 3H: 31

Number of DE genes definatetively trans regulated and on 3H: 3
